In [ ]:
import tkinter as tk
from tkinter import filedialog
import speech_recognition as sr
import threading
import os

class VoiceRecorderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Voice Recorder")

        self.recording = False
        self.audio_data = []
        self.transcription = ""

        self.label = tk.Label(root, text="Press 'Start Recording' to begin")
        self.label.pack()

        self.text_widget = tk.Text(root, height=10, width=40)
        self.text_widget.pack()

        self.start_button = tk.Button(root, text="Start Recording", command=self.start_recording)
        self.start_button.pack()

        self.stop_button = tk.Button(root, text="Stop Recording", state=tk.DISABLED, command=self.stop_recording)
        self.stop_button.pack()

        self.save_button = tk.Button(root, text="Save as Text", state=tk.DISABLED, command=self.save_transcription)
        self.save_button.pack()

        self.save_directory = None

        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()

    def start_recording(self):
        if not self.recording:
            self.recording = True
            self.audio_data = []
            self.label.config(text="Recording... Press 'Stop Recording' to finish")
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.save_button.config(state=tk.DISABLED)
            self.text_widget.delete(1.0, tk.END)  # Clear previous text

            self.audio_data.append("User: Yes")

            self.audio_thread = threading.Thread(target=self.record_audio)
            self.audio_thread.start()

    def stop_recording(self):
        if self.recording:
            self.recording = False
            self.label.config(text="Recording stopped")
            self.start_button.config(state=tk.NORMAL)
            self.stop_button.config(state=tk.DISABLED)
            self.save_button.config(state=tk.NORMAL)

            self.audio_data.append("User: No")

    def record_audio(self):
        with self.microphone as source:
            try:
                while self.recording:
                    audio = self.recognizer.listen(source)
                    self.audio_data.append(audio)

            except Exception as e:
                print("Error:", e)

    def save_transcription(self):
        if self.audio_data:
            with filedialog.asksaveasfilename(defaultextension=".txt") as file_path:
                if file_path:
                    with open(file_path, "w") as file:
                        self.transcription = self.transcribe_audio()
                        file.write(self.transcription)

    def transcribe_audio(self):
        audio = " ".join(self.audio_data)
        try:
            audio_text = self.recognizer.recognize_google(audio)
            self.text_widget.delete(1.0, tk.END)  # Clear previous text
            self.text_widget.insert(tk.END, audio_text)
            return audio_text
        except sr.UnknownValueError:
            self.text_widget.delete(1.0, tk.END)
            self.text_widget.insert(tk.END, "No speech detected")
            return ""

if __name__ == "__main__":
    root = tk.Tk()
    app = VoiceRecorderApp(root)
    root.mainloop()